# SquadBeyond 自動化ツール

## 使い方
1. **セル1** を実行 → セットアップ（初回のみ1〜2分かかります）
2. **セル2** を実行 → Google Driveからアカウント情報を読み込み
3. **セル3** を実行 → 自動化ロジックの準備
4. **セル4** を実行 → アカウント選択・CSVファイルアップロード・実行モード選択
5. **セル5** を実行 → 処理開始

**上から順番に全てのセルを実行してください。**

In [ ]:
#@title **セル1: セットアップ**（初回は1〜2分かかります）

# Google Chrome 安定版を直接インストール（Colabのchromiumはsnap版で動かないため）
!apt-get update -qq > /dev/null
!apt-get install -qq libu2f-udev libvulkan1 > /dev/null 2>&1
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb > /dev/null 2>&1
!apt-get -f install -qq > /dev/null 2>&1

# 必要なパッケージのインストール
!pip install -q selenium pandas chromedriver-autoinstaller

# ライブラリの読み込み
import time
import os
import io
import math
import logging
import concurrent.futures
import threading
import pandas as pd
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# ChromeDriverをChromeバージョンに合わせて自動インストール
chromedriver_autoinstaller.install()

# urllib3の警告ログを非表示にする
logging.getLogger('urllib3').setLevel(logging.ERROR)
logging.getLogger('selenium').setLevel(logging.ERROR)

print("✅ セットアップ完了")

In [ ]:
#@title **セル2: アカウント情報の読み込み**
#@markdown Googleアカウントでログインしてください（ポップアップが出ます）

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io as _io

auth.authenticate_user()

service = build('drive', 'v3')
request = service.files().get_media(fileId='12cYadGtLVyV9mJwsYNH2o9xalmJmHi1r')
fh = _io.FileIO('accounts.csv', 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while not done:
    status, done = downloader.next_chunk()

df_acc = pd.read_csv('accounts.csv')
ACCOUNTS = {}
for i, row in df_acc.iterrows():
    key = str(i + 1)
    ACCOUNTS[key] = {
        "name": str(row["name"]),
        "id": str(row["id"]),
        "pass": str(row["pass"]),
    }
print("✅ アカウント情報を読み込みました")

In [ ]:
#@title **セル3: 自動化ロジックの準備**

# === 定数 ===
LOGIN_URL = 'https://app.squadbeyond.com/'
BASE_URL = 'https://app.squadbeyond.com/'
MAX_WORKERS = 3

# === ログアウト検知用フラグ（全ワーカー共有） ===
_logged_out = threading.Event()

# === ポート管理（ワーカーごとに異なるポートを割り当て） ===
_port_counter = threading.Lock()
_next_port = [9222]

def _get_next_port():
    with _port_counter:
        port = _next_port[0]
        _next_port[0] += 1
        return port

# === 進捗カウンター（全ワーカー共有） ===
_progress_lock = threading.Lock()
_completed_count = [0]
_total_count = [0]

def _log_progress(log_func, msg):
    with _progress_lock:
        log_func(msg)

def _log_step(log_func, step_name, item_num=None):
    if item_num is not None:
        with _progress_lock:
            total = _total_count[0]
        _log_progress(log_func, f'  [{item_num}/{total}] {step_name}')
    else:
        _log_progress(log_func, f'  {step_name}')

def _log_complete(log_func, item_num, url):
    with _progress_lock:
        _completed_count[0] += 1
        done = _completed_count[0]
        total = _total_count[0]
    log_func(f'✅ [{done}/{total}] 完了: {url}')

# === スクリーンショット保存 ===
def _save_screenshot(driver, item_num, step_name, save_enabled):
    if not save_enabled:
        return
    try:
        fname = f'/content/screenshots/{item_num:03d}_{step_name}.png'
        driver.save_screenshot(fname)
    except:
        pass

# === HTML構造をダンプ（デバッグ用） ===
def _save_html(driver, item_num, step_name, save_enabled):
    if not save_enabled:
        return
    try:
        fname = f'/content/screenshots/{item_num:03d}_{step_name}.html'
        with open(fname, 'w', encoding='utf-8') as f:
            f.write(driver.page_source)
    except:
        pass

# === Chrome設定 ===
def get_chrome_options(port=None):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-software-rasterizer')
    if port:
        options.add_argument(f'--remote-debugging-port={port}')
    return options

# === Chromeドライバーを生成 ===
def create_driver():
    port = _get_next_port()
    options = get_chrome_options(port)
    return webdriver.Chrome(options=options)

# === ログアウトされたかチェック ===
def is_logged_out(driver):
    try:
        page_text = driver.execute_script("return document.body.innerText || '';")
        if 'ログインセッションの有効期限が過ぎました' in page_text:
            return True
        if 'ログインページへ' in page_text and 'トップページからログイン' in page_text:
            return True
    except:
        pass
    return False

# === ログインしてセッションCookieを取得 ===
def get_session_cookies(account_info, log_func, save_ss=False):
    log_func('🔐 ログイン中...')

    driver = create_driver()
    wait = WebDriverWait(driver, 20)
    cookies = []

    try:
        driver.get(LOGIN_URL)
        time.sleep(2)
        _save_screenshot(driver, 0, '01_login_page', save_ss)

        email_xpath = "//input[@id=':r1:'] | //input[@name='email']"
        email_input = wait.until(EC.element_to_be_clickable((By.XPATH, email_xpath)))
        email_input.clear()
        email_input.send_keys(account_info['id'])

        pass_xpath = "//input[@id=':r2:'] | //input[@type='password']"
        pass_input = wait.until(EC.element_to_be_clickable((By.XPATH, pass_xpath)))
        pass_input.clear()
        pass_input.send_keys(account_info['pass'])
        _save_screenshot(driver, 0, '02_credentials_entered', save_ss)

        btn1_xpath = "//button[@data-trackid='sign-in-form-login-button']"
        wait.until(EC.element_to_be_clickable((By.XPATH, btn1_xpath))).click()
        time.sleep(3)
        _save_screenshot(driver, 0, '03_after_first_login_click', save_ss)

        btn2_xpath = "//button[text()='ログイン' and contains(@class, 'MuiButton-sizeSmall')]"
        wait.until(EC.element_to_be_clickable((By.XPATH, btn2_xpath))).click()
        time.sleep(5)
        _save_screenshot(driver, 0, '04_after_second_login_click', save_ss)

        cookies = driver.get_cookies()
        log_func('✅ ログイン成功')

    except Exception as e:
        _save_screenshot(driver, 0, '99_login_error', save_ss)
        log_func('❌ ログインに失敗しました')
    finally:
        try:
            driver.quit()
        except:
            pass

    return cookies

# === 自動化ワーカー ===
def run_automation_worker(worker_id, df_subset, cookies, log_func, reset_rates, save_ss=False):
    if _logged_out.is_set():
        return

    driver = None
    try:
        driver = create_driver()
        wait = WebDriverWait(driver, 20)

        driver.get(BASE_URL)

        if not cookies:
            return

        for cookie in cookies:
            if 'expiry' in cookie:
                cookie['expiry'] = int(cookie['expiry'])
            driver.add_cookie(cookie)

        driver.refresh()
        time.sleep(3)

        for index, row in df_subset.iterrows():
            if _logged_out.is_set():
                return

            col1_url = str(row.iloc[0]) if pd.notna(row.iloc[0]) else ''
            col2_ver = str(row.iloc[1]) if pd.notna(row.iloc[1]) else ''
            col3_chk = str(row.iloc[2]) if pd.notna(row.iloc[2]) else ''
            col4_key = str(row.iloc[3]) if pd.notna(row.iloc[3]) else ''
            col5_lbl = str(row.iloc[4]) if pd.notna(row.iloc[4]) else ''
            col7_url = str(row.iloc[6]) if pd.notna(row.iloc[6]) else ''

            if not col1_url or col1_url == 'nan' or col1_url == '':
                continue

            item_num = index + 1
            _log_step(log_func, f'📂 ページを開いています...', item_num)
            driver.get(col1_url)
            time.sleep(3)
            _save_screenshot(driver, item_num, '01_page_opened', save_ss)

            if is_logged_out(driver):
                _logged_out.set()
                _save_screenshot(driver, item_num, '99_logged_out', save_ss)
                log_func('🚫 ログアウトされました。処理を停止します。')
                return

            # --- バージョン選択 ---
            _log_step(log_func, f'🔍 バージョン「{col2_ver}」を選択中...', item_num)
            try:
                xpath_ver = f"//span[@data-test='ArticleList-Memo' and contains(text(), '{col2_ver}')]"
                ver_elem = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_ver)))
                ver_elem.click()
                time.sleep(2)
                _save_screenshot(driver, item_num, '02_version_selected', save_ss)
            except Exception as e:
                _save_screenshot(driver, item_num, '02_version_not_found', save_ss)
                _save_html(driver, item_num, '02_version_not_found', save_ss)
                if is_logged_out(driver):
                    _logged_out.set()
                    log_func('🚫 ログアウトされました。処理を停止します。')
                    return
                _log_step(log_func, f'⚠️ バージョン「{col2_ver}」が見つかりません。スキップします', item_num)
                continue

            # --- 三点メニューを開く ---
            _log_step(log_func, f'📋 三点メニューを開いています...', item_num)
            menu_clicked = False

            # 方法1: CurrentArticle内のボタン（三点アイコン = M58,32のSVG path）
            try:
                menu_clicked = driver.execute_script("""
                    var current = document.querySelector('[data-test="ArticleList-CurrentArticle"]');
                    if (!current) return false;
                    var buttons = current.querySelectorAll('button');
                    for (var btn of buttons) {
                        var svg = btn.querySelector('svg');
                        if (svg) {
                            var path = svg.querySelector('path');
                            if (path) {
                                var d = path.getAttribute('d') || '';
                                if (d.includes('M58,32') || d.includes('M32,28') || d.includes('M19,28') || d.includes('M45,28')) {
                                    btn.click();
                                    return true;
                                }
                            }
                        }
                    }
                    return false;
                """)
                if menu_clicked:
                    _log_step(log_func, f'  → メニュー発見（CurrentArticle内SVG三点アイコン）', item_num)
            except:
                menu_clicked = False

            # 方法2: CurrentArticle内の最初のbuttonをクリック（更新ボタンは2番目）
            if not menu_clicked:
                try:
                    menu_clicked = driver.execute_script("""
                        var current = document.querySelector('[data-test="ArticleList-CurrentArticle"]');
                        if (!current) return false;
                        var btnsDiv = current.querySelector('[class*="articleButtons"]');
                        if (!btnsDiv) return false;
                        var firstBtn = btnsDiv.querySelector('button');
                        if (firstBtn) {
                            firstBtn.click();
                            return true;
                        }
                        return false;
                    """)
                    if menu_clicked:
                        _log_step(log_func, f'  → メニュー発見（articleButtons内の最初のボタン）', item_num)
                except:
                    menu_clicked = False

            # 方法3: CSSクラス css-3tls8 のボタン（CurrentArticle内）
            if not menu_clicked:
                try:
                    menu_clicked = driver.execute_script("""
                        var current = document.querySelector('[data-test="ArticleList-CurrentArticle"]');
                        if (!current) return false;
                        var btn = current.querySelector('button.MuiButtonBase-root:not(.MuiButton-root)');
                        if (btn) {
                            btn.click();
                            return true;
                        }
                        return false;
                    """)
                    if menu_clicked:
                        _log_step(log_func, f'  → メニュー発見（MuiButtonBase非MuiButton）', item_num)
                except:
                    menu_clicked = False

            time.sleep(1)
            _save_screenshot(driver, item_num, '03_menu_opened', save_ss)
            _save_html(driver, item_num, '03_after_menu_click', save_ss)

            if not menu_clicked:
                _log_step(log_func, f'❌ 三点メニューが見つかりません。スキップします', item_num)
                continue

            # --- 「別のbeyondページに複製」をクリック ---
            _log_step(log_func, f'📋 「別のbeyondページに複製」をクリック中...', item_num)
            clone_clicked = False
            try:
                clone_menu_xpath = "//span[contains(text(), '別のbeyondページに複製')]"
                clone_elem = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, clone_menu_xpath)))
                clone_elem.click()
                clone_clicked = True
            except:
                pass

            if not clone_clicked:
                try:
                    clone_clicked = driver.execute_script("""
                        var items = document.querySelectorAll('li, span, div, p, a');
                        for (var e of items) {
                            var txt = e.textContent.trim();
                            if (txt === '別のbeyondページに複製') {
                                e.click();
                                return true;
                            }
                        }
                        return false;
                    """)
                except:
                    clone_clicked = False

            if not clone_clicked:
                _save_screenshot(driver, item_num, '03b_clone_menu_not_found', save_ss)
                _save_html(driver, item_num, '03b_clone_menu_not_found', save_ss)
                _log_step(log_func, f'❌ 「別のbeyondページに複製」が見つかりません。スキップします', item_num)
                try:
                    driver.execute_script("document.body.click();")
                except:
                    pass
                continue

            time.sleep(2)
            _save_screenshot(driver, item_num, '04_clone_dialog', save_ss)

            # --- 複製先を選択 ---
            _log_step(log_func, f'📋 複製先「{col5_lbl}」を選択中...', item_num)
            clone_done = False
            try:
                select_xpath = "//select[@data-test='DuplicateToOtherModal-DestinationAbTestUidSelect']"
                select_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, select_xpath)))
                Select(select_elem).select_by_visible_text(col5_lbl)
                _save_screenshot(driver, item_num, '05_destination_selected', save_ss)

                btn_xpath = "//div[contains(text(), '複製する')]"
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, btn_xpath))).click()
                clone_done = True
                _log_step(log_func, f'✅ 複製実行しました', item_num)
            except Exception as e:
                _save_screenshot(driver, item_num, '05_destination_error', save_ss)
                _save_html(driver, item_num, '05_destination_error', save_ss)
                _log_step(log_func, f'❌ 複製先の選択に失敗: {str(e)[:100]}', item_num)

            time.sleep(3)
            _save_screenshot(driver, item_num, '06_after_clone', save_ss)

            if is_logged_out(driver):
                _logged_out.set()
                log_func('🚫 ログアウトされました。処理を停止します。')
                return

            # --- 配信割合設定 ---
            _log_step(log_func, f'📊 配信割合を設定中...', item_num)
            try:
                rate_xpath = f"//input[@value='{col4_key}']/following::div[@data-test='ArticleList-DeriveryUpRateForm'][1]"
                wait.until(EC.element_to_be_clickable((By.XPATH, rate_xpath))).click()
            except Exception as e:
                _log_step(log_func, f'  ⚠️ 配信割合ボタン: {str(e)[:80]}', item_num)

            if reset_rates:
                _log_step(log_func, f'📊 他のバージョンを0%に設定中...', item_num)
                js_rate_0 = """
                var others = document.querySelectorAll("div[data-test='ArticleList-Article'] input[data-test='ArticleList-DeriveryRateForm']");
                var setter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, "value").set;
                others.forEach(function(i){
                    setter.call(i, "0");
                    i.dispatchEvent(new Event('input', { bubbles: true }));
                });
                """
                driver.execute_script(js_rate_0)
            _save_screenshot(driver, item_num, '07_rates_set', save_ss)

            # --- リンク置換 ---
            _log_step(log_func, f'🔗 リンク置換中...', item_num)
            try:
                link_replace_xpath = '//*[@id="root"]/div[2]/div/div[4]/div[7]/div/div[4]/div/div/div/div/div/img'
                time.sleep(1)
                replace_icon = wait.until(EC.element_to_be_clickable((By.XPATH, link_replace_xpath)))
                replace_icon.click()
                time.sleep(1)
            except Exception as e:
                _log_step(log_func, f'  ⚠️ リンク置換アイコン: {str(e)[:80]}', item_num)

            try:
                chk_xpath = f"//div[contains(text(), '{col3_chk}')]"
                chk_elem = wait.until(EC.element_to_be_clickable((By.XPATH, chk_xpath)))
                chk_elem.click()
            except Exception as e:
                _log_step(log_func, f'  ⚠️ チェックボックス: {str(e)[:80]}', item_num)

            try:
                input_xpath = "//input[@placeholder='新規のリンクを入力']"
                input_elem = wait.until(EC.presence_of_element_located((By.XPATH, input_xpath)))
                driver.execute_script("""
                    var element = arguments[0];
                    var value = arguments[1];
                    var setter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, "value").set;
                    setter.call(element, value);
                    element.dispatchEvent(new Event('input', { bubbles: true }));
                """, input_elem, col7_url)
            except Exception as e:
                _log_step(log_func, f'  ⚠️ リンク入力: {str(e)[:80]}', item_num)
            time.sleep(1)
            _save_screenshot(driver, item_num, '08_link_entered', save_ss)

            try:
                replace_btn_xpath = '//*[@id="root"]/div[2]/div/div[4]/div[7]/div/div[4]/div/div[2]/div/div/div[2]/div[5]/div/div[2]'
                wait.until(EC.element_to_be_clickable((By.XPATH, replace_btn_xpath))).click()
            except:
                driver.execute_script("var b=Array.from(document.querySelectorAll('div')).find(e=>e.textContent=='置換');if(b)b.click();")
            time.sleep(1)
            _save_screenshot(driver, item_num, '09_link_replaced', save_ss)

            # --- ページ更新 ---
            _log_step(log_func, f'💾 ページを更新中...', item_num)
            driver.refresh()
            time.sleep(2)
            _save_screenshot(driver, item_num, '10_page_refreshed', save_ss)

            if is_logged_out(driver):
                _logged_out.set()
                log_func('🚫 ログアウトされました。処理を停止します。')
                return

            try:
                update_btn_xpath = "//button[text()='更新']"
                wait.until(EC.element_to_be_clickable((By.XPATH, update_btn_xpath))).click()
                target_class_fragment = 'css-1g290g6'
                success_condition_xpath = f"//button[text()='更新' and contains(@class, '{target_class_fragment}')]"
                wait.until(EC.presence_of_element_located((By.XPATH, success_condition_xpath)))
                time.sleep(0.5)
            except Exception as e:
                try:
                    driver.execute_script("""
                    var btns = document.querySelectorAll("button");
                    var target = Array.from(btns).find(b => b.textContent.trim() === "更新");
                    if(target){ target.click(); }
                    """)
                    time.sleep(2)
                except:
                    pass
            _save_screenshot(driver, item_num, '11_page_updated', save_ss)

            # --- ポップアップ設定 ---
            _log_step(log_func, f'🔔 ポップアップ設定中...', item_num)
            try:
                current_url = driver.current_url
                if '#' in current_url:
                    base_url = current_url.split('#')[0]
                else:
                    base_url = current_url.rstrip('/').rsplit('/', 1)[0]
                target_url = base_url + '/exit_popups'
                driver.get(target_url)
                time.sleep(2)
            except Exception as e:
                _log_step(log_func, f'  ⚠️ ポップアップページ遷移: {str(e)[:80]}', item_num)
            _save_screenshot(driver, item_num, '12_popup_page', save_ss)

            try:
                target_xpath = f"//h6[contains(text(), '{col4_key}')]"
                wait.until(EC.element_to_be_clickable((By.XPATH, target_xpath))).click()
            except:
                pass

            try:
                base_xpath = "//span[@aria-label='ONにすると選択中のVersionにポップアップ/バナーが紐づきます']"
                wait.until(EC.presence_of_element_located((By.XPATH, base_xpath)))
                driver.execute_script("""
                var parentSpan = document.querySelector("span[aria-label='ONにすると選択中のVersionにポップアップ/バナーが紐づきます']");
                var input = parentSpan.querySelector("input");
                if(input && !input.checked){
                    input.click();
                }
                """)
                on_state_xpath = base_xpath + "[contains(@class, 'Mui-checked')]"
                wait.until(EC.presence_of_element_located((By.XPATH, on_state_xpath)))
            except Exception as e:
                pass
            _save_screenshot(driver, item_num, '13_popup_toggled', save_ss)

            _log_step(log_func, f'🌐 ポップアップURL設定中...', item_num)
            try:
                url_xpath = "//input[@name='urlControl']"
                url_input = wait.until(EC.element_to_be_clickable((By.XPATH, url_xpath)))
                current_val = url_input.get_attribute('value')
                if current_val != col7_url:
                    driver.execute_script("""
                        var element = arguments[0];
                        var value = arguments[1];
                        var setter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, "value").set;
                        setter.call(element, value);
                        element.dispatchEvent(new Event('input', { bubbles: true }));
                    """, url_input, col7_url)
                    time.sleep(1)
                    wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()='本番反映']"))).click()
            except Exception as e:
                pass
            _save_screenshot(driver, item_num, '14_popup_url_set', save_ss)

            _log_complete(log_func, item_num, col1_url)
            time.sleep(1)

    except Exception as e:
        try:
            if driver:
                _save_screenshot(driver, 0, '99_worker_error', save_ss)
            if driver and is_logged_out(driver):
                _logged_out.set()
                log_func('🚫 ログアウトされました。処理を停止します。')
            else:
                log_func(f'❌ エラーが発生しました: {str(e)[:100]}')
        except:
            log_func(f'❌ エラーが発生しました')
    finally:
        try:
            if driver:
                driver.quit()
        except:
            pass

print('✅ 準備完了')

In [ ]:
#@title **セル4: 実行設定**

from google.colab import files

# アカウント一覧を表示（番号・名前・メールアドレス）
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print(' 番号  名前                    メールアドレス')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
for k, v in ACCOUNTS.items():
    print(f'  {k}    {v["name"]:<24}{v["id"]}')
print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
print()

account_number = input('アカウント番号: ')
if account_number not in ACCOUNTS:
    print(f'❌ アカウント番号 {account_number} は存在しません')
else:
    selected_account = ACCOUNTS[account_number]
    print(f'→ {selected_account["name"]}（{selected_account["id"]}）')
    print()

    print('CSVファイルをアップロードしてください:')
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    csv_data = uploaded[filename].decode('utf-8')
    print(f'→ {filename} を読み込みました')
    print()

    print('他を0%にする？')
    mode = input('1=はい / 0=いいえ: ')
    reset_rates = (mode == '1')
    print(f'→ {"他を0%にする" if reset_rates else "割合変更なし"}')
    print()

    print('スクショ保存する？（操作確認用）')
    dbg = input('1=はい / 0=いいえ: ')
    debug_screenshots = (dbg == '1')
    if debug_screenshots:
        os.makedirs('/content/screenshots', exist_ok=True)
        print('→ スクショON（左のファイル📁 → screenshots/で確認）')
    else:
        print('→ スクショOFF')
    print()
    print('✅ 設定完了。次のセルを実行してください。')

In [ ]:
#@title **セル5: 実行**

import shutil

# ログアウトフラグ・進捗カウンターをリセット
_logged_out.clear()
_completed_count[0] = 0
_next_port[0] = 9222

# 前回のスクショを削除
if os.path.exists('/content/screenshots'):
    shutil.rmtree('/content/screenshots')
if debug_screenshots:
    os.makedirs('/content/screenshots', exist_ok=True)

try:
    df = pd.read_csv(io.StringIO(csv_data), header=None)
except Exception as e:
    print('❌ CSVファイルの読み込みに失敗しました')
    df = None

if df is not None:
    total_rows = len(df)
    _total_count[0] = total_rows
    print(f'📄 処理対象: {total_rows} 件')
    print()

    cookies = get_session_cookies(selected_account, print, save_ss=debug_screenshots)

    if not cookies:
        print('❌ ログインに失敗しました。アカウント情報を確認してください。')
    else:
        print(f'🚀 {total_rows} 件の処理を開始します（並列数: {MAX_WORKERS}）')
        print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
        print()

        chunk_size = math.ceil(total_rows / MAX_WORKERS)
        chunks = [df.iloc[i:i + chunk_size] for i in range(0, total_rows, chunk_size)]

        try:
            with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                futures = []
                for i, df_subset in enumerate(chunks):
                    if i > 0:
                        time.sleep(5)
                    futures.append(
                        executor.submit(run_automation_worker, i + 1, df_subset, cookies, print, reset_rates, save_ss=debug_screenshots)
                    )

                # タイムアウト付きで待機（KeyboardInterrupt対策）
                done, not_done = concurrent.futures.wait(futures, timeout=None)

                # 各futureの例外をチェック
                for f in done:
                    try:
                        f.result()
                    except Exception:
                        pass

        except KeyboardInterrupt:
            print()
            print('⚠️ 中断されました')
        except Exception:
            pass

        print()
        print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')
        completed = _completed_count[0]
        if _logged_out.is_set():
            print(f'🚫 ログアウトされたため途中で停止しました（{completed}/{total_rows} 件完了）')
        elif completed == total_rows:
            print(f'✅ 全 {total_rows} 件の処理が完了しました')
        else:
            print(f'⚠️ {completed}/{total_rows} 件完了（一部スキップまたはエラーあり）')
        print('━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━')

    # クリーンアップ: アップロードしたCSVを削除
    try:
        if os.path.exists(filename):
            os.remove(filename)
    except:
        pass